## demo mcmc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model1 as model

%matplotlib inline

## define functions and constants

In [2]:
def newtheta(theta):
    a = 0.05
    fa = lambda x:x + a*x*np.random.randn()
    current = theta
    #newkc = mapkc(current.KC.copy())
    #newsig = mapkc(current.sig.copy())
    proposed = current.map(fa)
    #proposed = current.drop(['KC','sig']).map(fa)
    #proposed['KC']=newkc
    #proposed['sigma']=newsig
    return proposed

In [3]:
def chisq(data,model,variance):
    d = data
    m = model
    v = variance

    #for first epoch probability
    fmax = lambda x:(max(x,0.001)**2)
    N1 = m['COUNTS_U1']
    x1 = (m['FIRST_EPOCH_PROB_U1']-d['FIRST_EPOCH_PROB_U1'])**2
    s1 = m['FIRST_EPOCH_PROB_U1'].map(fmax)
    N2 = m['COUNTS_U2']
    x2 = (m['FIRST_EPOCH_PROB_U2']-d['FIRST_EPOCH_PROB_U2'])**2
    s2 = m['FIRST_EPOCH_PROB_U2'].map(fmax)

    #other chi's follow form: chi = ((d-m)**2)/(v**2)
    chi = pd.DataFrame()
    chi['FIRST_EPOCH_PROB_U1'] = (N1*(x1/s1) + N2*(x2/s2))
    chi['MEAN_U1'] = ((m['MEAN_U1']-d['MEAN_U1'])**2)/(v['MEAN_U1']**2)
    chi['MEAN_U2'] = ((m['MEAN_U2']-d['MEAN_U2'])**2)/(v['MEAN_U2']**2)
    chi['SD_U1'] = ((m['SD_U1']-d['SD_U1'])**2)/(v['SD_U1']**2)
    chi['SD_U2'] = ((m['SD_U2']-d['SD_U2'])**2)/(v['SD_U2']**2)

    return chi

In [4]:
def chisum(df):
    #return scalar
    one = df.sum(axis=0)
    two = one.sum(0)
    return two

In [5]:
def taupen(tau_a):
    penalty = (tau_a-2)**2
    return penalty

def kpen(ks,kc):
    delta = 0
    pen = delta*(ks + kc)**2
    return pen

In [6]:
index = [0,20,40,60,80]
cv = 0.6
cv_var = 0.2
p1_var = 0.1
fcv = lambda x: x*cv
fcv_var = lambda x:(x*cv_var)
fp1_var = lambda x:(x*p1_var)

## load data

In [7]:
syprob = pd.read_csv('sy_prob.csv',index_col=0)
syU1 = pd.read_csv('syU1.csv',index_col=0)
syU2 = pd.read_csv('syU2.csv',index_col=0)

## construct dataframe of data

In [8]:
data = pd.DataFrame()
data['COUNTS_U1'] = syprob['counts_U1']
data['COUNTS_U2'] = syprob['counts_U2']
data['FIRST_EPOCH_PROB_U1'] = syprob['U1']
data['FIRST_EPOCH_PROB_U2'] = syprob['U2']
data['MEAN_U1'] = syU1['MEAN']
data['MEAN_U2'] = syU2['MEAN']
data['SD_U1'] = data['MEAN_U1'].map(fcv)
data['SD_U2'] = data['MEAN_U2'].map(fcv)
data.index=index
col = data.columns

In [9]:
data

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,50,50,0.50,0.50,1800.0,1700.0,1080.0,1020.0
20,95,5,0.95,0.05,1900.0,1700.0,1140.0,1020.0
40,99,1,0.99,0.01,2200.0,1700.0,1320.0,1020.0
60,100,0,1.00,0.00,2700.0,1700.0,1620.0,1020.0
80,100,0,1.00,0.00,3200.0,1700.0,1920.0,1020.0


## calculate error and make dataframe from data

In [10]:
variance = pd.DataFrame()
variance['COUNTS_U1'] = data['COUNTS_U1'].map(fp1_var)
variance['COUNTS_U2'] = data['COUNTS_U2'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U1'] = data['FIRST_EPOCH_PROB_U1'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U2'] = data['FIRST_EPOCH_PROB_U2'].map(fp1_var)
variance['MEAN_U1'] = data['MEAN_U1'].map(fcv_var)
variance['MEAN_U2'] = data['MEAN_U2'].map(fcv_var)
variance['SD_U1'] = data['SD_U1'].map(fcv_var)
variance['SD_U2'] = data['SD_U2'].map(fcv_var)
variance.index = index
variance.columns = col

In [11]:
variance

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,5.0,5.0,0.050,0.050,360.0,340.0,216.0,204.0
20,9.5,0.5,0.095,0.005,380.0,340.0,228.0,204.0
40,9.9,0.1,0.099,0.001,440.0,340.0,264.0,204.0
60,10.0,0.0,0.100,0.000,540.0,340.0,324.0,204.0
80,10.0,0.0,0.100,0.000,640.0,340.0,384.0,204.0


### load initial parameters

In [12]:
!ls

demodynamics.ipynb model1.py          syU1.csv           trex.pkl
demomcmc.ipynb     model1.pyc         syU2.csv
demomodel.ipynb    rivalry1.py        sy_bias.csv
mcmc.py            rivalry1.pyc       sy_prob.csv


In [13]:
initial = pd.read_pickle('trex.pkl')
initial

beta        1.700000
gamma       3.520000
kb          0.109369
kc          0.100000
ks          3.000000
mu          0.000000
sigma       0.400000
tau_a    1248.000000
tau_u      10.000000
dtype: float64

In [14]:
theta_cur = initial
theta_prop = newtheta(theta_cur)

In [15]:
print theta_cur
print theta_prop

beta        1.700000
gamma       3.520000
kb          0.109369
kc          0.100000
ks          3.000000
mu          0.000000
sigma       0.400000
tau_a    1248.000000
tau_u      10.000000
dtype: float64
beta        1.581468
gamma       3.498035
kb          0.108795
kc          0.098424
ks          3.025380
mu          0.000000
sigma       0.364512
tau_a    1251.472646
tau_u       9.841504
dtype: float64


## run model with current and proposed theta values

In [16]:
#trials
dominance = 1
firstprob = 200

In [17]:
m1 = model.run(theta_cur,dominance,firstprob)

In [18]:
m2 = model.run(theta_prop,dominance,firstprob)

In [27]:
m1.index = index
m1.columns = col
#m1['CV U1'] = m1['SD_U1']/m1['MEAN_U1']
#m1['CV U2'] = m1['SD_U2']/m1['MEAN_U2']
m1

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2,CV U1,CV U2
0,103.0,97.0,0.515,0.485,208.554357,208.211706,145.631635,145.371687,0.698291,0.698192
20,74.0,126.0,0.370,0.630,225.322855,201.113424,153.233565,136.345871,0.680062,0.677955
40,200.0,0.0,1.000,0.000,233.701593,186.914956,164.239296,130.862031,0.702774,0.700115
60,200.0,0.0,1.000,0.000,246.974283,176.114201,176.475757,125.238880,0.714551,0.711123
80,200.0,0.0,1.000,0.000,265.826464,169.395490,188.178699,118.734680,0.707901,0.700932


In [21]:
m2.index = index
m2.columns=col
m2

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,97.0,103.0,0.485,0.515,182.923525,183.008012,136.143291,136.251759
20,108.0,92.0,0.540,0.460,198.462866,178.240810,142.998249,128.325588
40,160.0,40.0,0.800,0.200,209.159910,167.650901,151.589427,120.849097
60,200.0,0.0,1.000,0.000,215.836290,155.316704,163.304175,116.539941
80,200.0,0.0,1.000,0.000,231.359245,147.689434,175.962955,111.426103


## caluclate chi construct dataframe

In [22]:
chicur = pd.DataFrame(chisq(data,m1,variance),index=index)
chiprop = pd.DataFrame(chisq(data,m2,variance),index=index)

In [23]:
chicur

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,0.180162,19.542432,19.251145,18.712368,18.381745
20,288.631489,19.422047,19.434783,18.730917,18.763087
40,0.020000,19.970710,19.804726,19.165846,18.996692
60,0.000000,20.635580,20.088477,19.849892,19.237732
80,0.000000,21.018980,20.266005,20.339661,19.518435


In [24]:
chiprop

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,0.180162,20.176978,19.907134,19.094339,18.767084
20,135.346216,20.050060,20.032448,19.121508,19.105230
40,45.125000,20.472336,20.312230,19.587685,19.426959
60,0.000000,21.162789,20.640541,20.213789,19.613612
80,0.000000,21.515693,20.844880,20.627612,19.836278


##  calulate likelihood ratio

In [25]:
sumchicur = chisum(chicur)
sumprop = chisum(chiprop)
print sumchicur
print sumprop

679.96291
581.160562856


In [26]:
ratio = np.exp((-sumprop+sumchicur)/2)
ratio

2.8487679972608552e+21